In [ ]:
import pandas as pd

def basic_clean(df: pd.DataFrame, required_cols=None, dtypes=None,
                drop_duplicates=True, drop_na_rows=False) -> pd.DataFrame:
    # Enforce required columns if provided
    if required_cols:
        missing = [c for c in required_cols if c not in df.columns]
        if missing:
            raise ValueError(f"Missing required columns: {missing}")

    # Cast dtypes if provided
    if dtypes:
        for col, dtype in dtypes.items():
            if col in df.columns:
                df[col] = df[col].astype(dtype, errors="ignore")

    # Deduplicate
    if drop_duplicates:
        df = df.drop_duplicates()

    # Drop NA rows if requested
    if drop_na_rows:
        df = df.dropna()

    return df


def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    # Example features (adapt to your domain)
    # Create normalized numeric features
    for col in df.select_dtypes(include="number").columns:
        max_val = df[col].max()
        min_val = df[col].min()
        if pd.notna(max_val) and pd.notna(min_val) and max_val != min_val:
            df[f"{col}_norm"] = (df[col] - min_val) / (max_val - min_val)

    # Example: timestamp to date, hour
    for col in df.columns:
        if "time" in col and "event" in col and df[col].dtype == "object":
            ts = pd.to_datetime(df[col], errors="coerce")
            df[f"{col}_date"] = ts.dt.date
            df[f"{col}_hour"] = ts.dt.hour

    return df


def transform_chunk(df: pd.DataFrame,
                    required_cols=None,
                    dtypes=None,
                    enable_features=True,
                    drop_duplicates=True,
                    drop_na_rows=False) -> pd.DataFrame:
    df = basic_clean(
        df,
        required_cols=required_cols,
        dtypes=dtypes,
        drop_duplicates=drop_duplicates,
        drop_na_rows=drop_na_rows
    )
    if enable_features:
        df = feature_engineering(df)
    return df